Note: we should define some terms (like maybe player-frame to refer to the tracking data since there's one entry for every player for every frame)

# Load data and combine tracking data:

In [25]:
import pandas as pd
import statistics as stats
import os
games = pd.read_csv('../input/nfl-big-data-bowl-2021/games.csv')
players = pd.read_csv('../input/nfl-big-data-bowl-2021/players.csv')
plays = pd.read_csv('../input/nfl-big-data-bowl-2021/plays.csv')
week1 = pd.read_csv('../input/nfl-big-data-bowl-2021/week1.csv')
week2 = pd.read_csv('../input/nfl-big-data-bowl-2021/week2.csv')
week3 = pd.read_csv('../input/nfl-big-data-bowl-2021/week3.csv')
week4 = pd.read_csv('../input/nfl-big-data-bowl-2021/week4.csv')
week5 = pd.read_csv('../input/nfl-big-data-bowl-2021/week5.csv')
week6 = pd.read_csv('../input/nfl-big-data-bowl-2021/week6.csv')
week7 = pd.read_csv('../input/nfl-big-data-bowl-2021/week7.csv')
week8 = pd.read_csv('../input/nfl-big-data-bowl-2021/week8.csv')
week9 = pd.read_csv('../input/nfl-big-data-bowl-2021/week9.csv')
week10 = pd.read_csv('../input/nfl-big-data-bowl-2021/week10.csv')
week11 = pd.read_csv('../input/nfl-big-data-bowl-2021/week11.csv')
week12 = pd.read_csv('../input/nfl-big-data-bowl-2021/week12.csv')
week13 = pd.read_csv('../input/nfl-big-data-bowl-2021/week13.csv')
week14 = pd.read_csv('../input/nfl-big-data-bowl-2021/week14.csv')
week15 = pd.read_csv('../input/nfl-big-data-bowl-2021/week15.csv')
week16 = pd.read_csv('../input/nfl-big-data-bowl-2021/week16.csv')
week17 = pd.read_csv('../input/nfl-big-data-bowl-2021/week17.csv')
targeted_receiver = pd.read_csv('../input/nfl-big-data-bowl-2021-bonus/targetedReceiver.csv')

# TODO: add coverage data?

frames = [week1, week2, week3, week4, week5, week6, week7, week8, week9, week10, week11, week12, week13, week14, week15, week16, week17]
tracking_original = pd.concat(frames, ignore_index = True)
tracking_original

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,gameId,playId,playDirection,route
0,2018-09-07T01:07:14.599Z,91.73,26.67,0.00,0.01,0.02,289.57,240.93,None,310.0,Matt Ryan,2.0,QB,1,away,2018090600,75,left,NaN
1,2018-09-07T01:07:14.599Z,88.89,36.47,0.01,0.01,0.01,105.63,66.66,None,79848.0,Malcolm Jenkins,27.0,SS,1,home,2018090600,75,left,NaN
2,2018-09-07T01:07:14.599Z,91.35,44.16,0.02,0.03,0.01,290.45,16.86,None,2495454.0,Julio Jones,11.0,WR,1,away,2018090600,75,left,HITCH
3,2018-09-07T01:07:14.599Z,86.31,22.01,0.09,0.42,0.01,70.12,168.91,None,2495613.0,Corey Graham,24.0,FS,1,home,2018090600,75,left,NaN
4,2018-09-07T01:07:14.599Z,90.78,36.15,0.00,0.00,0.00,257.61,193.97,None,2533040.0,Mohamed Sanu,12.0,WR,1,away,2018090600,75,left,HITCH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18309383,2018-12-31T04:23:24.200Z,95.77,11.20,7.15,1.75,0.73,103.97,118.54,None,2558942.0,Anthony Firkser,86.0,TE,102,home,2018123011,4261,left,POST
18309384,2018-12-31T04:23:24.200Z,101.94,14.60,5.63,2.24,0.57,139.23,119.97,None,2558976.0,Kenny Moore,23.0,CB,102,away,2018123011,4261,left,NaN
18309385,2018-12-31T04:23:24.200Z,102.68,0.94,2.34,1.67,0.24,164.79,152.29,None,2560789.0,Darius Leonard,53.0,LB,102,away,2018123011,4261,left,NaN
18309386,2018-12-31T04:23:24.200Z,105.82,0.71,2.28,3.53,0.25,284.72,93.44,None,2561521.0,Cameron Batson,17.0,WR,102,home,2018123011,4261,left,OUT


# Reorient tracking data to all face the same way:

In [26]:
tracking_df = tracking_original.copy()
tracking_df.loc[tracking_df['playDirection'] == 'left', 'x'] = 120 - tracking_df['x']
tracking_df.loc[tracking_df['playDirection'] == 'left', 'y'] = 53.33 - tracking_df['y']
tracking_df.loc[(tracking_df['playDirection'] == 'left') & (tracking_original['o'] <= 180), 'o'] = tracking_df['o'] + 180
tracking_df.loc[(tracking_df['playDirection'] == 'left') & (tracking_original['o'] > 180), 'o'] = tracking_df['o'] - 180
tracking_df.loc[(tracking_df['playDirection'] == 'left') & (tracking_original['dir'] <= 180), 'dir'] = tracking_df['dir'] + 180
tracking_df.loc[(tracking_df['playDirection'] == 'left') & (tracking_original['dir'] > 180), 'dir'] = tracking_df['dir'] - 180

# Add week 1 coverages and targeted receiver for each play:

In [27]:
# Add week 1 coverages
# plays = pd.merge(plays, coverages_week1, how='left')

# Add targeted receiver for each play
plays = pd.merge(plays, targeted_receiver, how='left')
plays = pd.merge(plays, players, how='left', left_on='targetNflId', right_on='nflId')
plays.iloc[0]

gameId                                                           2018090600
playId                                                                   75
playDescription           (15:00) M.Ryan pass short right to J.Jones pus...
quarter                                                                   1
down                                                                      1
yardsToGo                                                                15
possessionTeam                                                          ATL
playType                                                     play_type_pass
yardlineSide                                                            ATL
yardlineNumber                                                           20
offenseFormation                                                     I_FORM
personnelO                                                 2 RB, 1 TE, 2 WR
defendersInTheBox                                                         7
numberOfPass

# Begin constructing plays_expanded, our pre-processed training data:

In [28]:
plays_expanded = plays[['gameId', 'playId', 'down', 'yardsToGo', 'penaltyCodes', 'penaltyJerseyNumbers', 'passResult', 'offensePlayResult', 'playResult', 'epa', 'nflId', 'displayName']]
plays_expanded

,gameId,playId,down,yardsToGo,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,nflId,displayName
0,2018090600,75,1,15,NaN,NaN,C,10,10,0.261827,2495454.0,Julio Jones
1,2018090600,146,1,10,NaN,NaN,I,0,0,-0.372360,2560854.0,Calvin Ridley
2,2018090600,168,2,10,NaN,NaN,I,0,0,-0.702779,2543583.0,Devonta Freeman
3,2018090600,190,3,10,NaN,NaN,C,33,33,3.047530,2495454.0,Julio Jones
4,2018090600,256,3,1,NaN,NaN,I,0,0,-0.842272,2543583.0,Devonta Freeman
...,...,...,...,...,...,...,...,...,...,...,...,...
19234,2018122200,2300,2,5,DH,TEN 24,I,0,5,0.848703,NaN,NaN
19235,2018122200,3177,3,7,DH,WAS 31,I,0,5,1.227695,2557870.0,Taywan Taylor
19236,2018122201,566,3,4,OPI,LAC 13,C,27,-10,-0.720168,2540154.0,Keenan Allen
19237,2018122201,1719,3,1,DPI,BAL 23,I,0,15,1.336078,2540154.0,Keenan Allen


# Filter tracking data for only the frames where ball is released:

In [29]:
def filter_tracking_frames_by_event(df, event_types):
    """
    General function for filtering tracking data by event type
    :param df: Pandas dataframe containing tracking data by frame
    :param event_types: List of event types to filter for
    :return: Pandas Dataframe containing frames corresponding to event_types
    """
    frame_indices = df['event'].isin(event_types)
    return df[frame_indices]

# Filter tracking data for the exact moments at which the ball is released for each play
release_df = filter_tracking_frames_by_event(tracking_df, ['pass_forward'])
release_df

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,gameId,playId,playDirection,route
490,2018-09-07T01:07:18.099Z,23.57,26.55,0.84,2.05,0.08,176.40,153.39,pass_forward,310.0,Matt Ryan,2.0,QB,36,away,2018090600,75,left,NaN
491,2018-09-07T01:07:18.099Z,35.18,15.00,5.53,4.30,0.54,115.34,138.58,pass_forward,79848.0,Malcolm Jenkins,27.0,SS,36,home,2018090600,75,left,NaN
492,2018-09-07T01:07:18.099Z,39.31,8.42,3.11,6.67,0.34,335.11,117.88,pass_forward,2495454.0,Julio Jones,11.0,WR,36,away,2018090600,75,left,HITCH
493,2018-09-07T01:07:18.099Z,33.98,34.69,3.52,0.69,0.35,208.19,36.15,pass_forward,2495613.0,Corey Graham,24.0,FS,36,home,2018090600,75,left,NaN
494,2018-09-07T01:07:18.099Z,37.35,18.77,6.35,1.21,0.64,82.06,92.78,pass_forward,2533040.0,Mohamed Sanu,12.0,WR,36,away,2018090600,75,left,HITCH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18308473,2018-12-31T04:23:17.700Z,50.43,36.35,7.30,4.98,0.73,166.46,108.21,pass_forward,2558942.0,Anthony Firkser,86.0,TE,37,home,2018123011,4261,left,POST
18308474,2018-12-31T04:23:17.700Z,48.51,17.14,2.84,3.67,0.29,281.51,78.99,pass_forward,2558976.0,Kenny Moore,23.0,CB,37,away,2018123011,4261,left,NaN
18308475,2018-12-31T04:23:17.700Z,47.68,25.85,2.45,4.25,0.24,289.79,354.67,pass_forward,2560789.0,Darius Leonard,53.0,LB,37,away,2018123011,4261,left,NaN
18308476,2018-12-31T04:23:17.700Z,48.34,43.77,6.66,5.27,0.67,3.83,37.07,pass_forward,2561521.0,Cameron Batson,17.0,WR,37,home,2018123011,4261,left,OUT


# Merge receiver tracking data with plays_expanded:

In [30]:
# TODO: Scott write some comments here about how this works
plays_expanded = pd.merge(plays_expanded, release_df, how='left', on = ['gameId', 'playId', 'nflId'], suffixes=('', '_y'))
plays_expanded.drop(plays_expanded.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
plays_expanded.drop(columns = ['time', 'dis', 'event'], inplace = True)
plays_expanded.rename(columns = {'team': 'team_o', 'nflId': 'nflId_o', 'displayName': 'displayName_o', 'x': 'x_o', 'y': 'y_o', 's': 'speed_o', 'a': 'acceleration_o', 'o': 'orientation_o', 'dir': 'dir_o', 'jerseyNumber': 'jerseyNumber_o', 'position': 'position_o'}, inplace = True)
plays_expanded.columns

Index(['gameId', 'playId', 'down', 'yardsToGo', 'penaltyCodes',
       'penaltyJerseyNumbers', 'passResult', 'offensePlayResult', 'playResult',
       'epa', 'nflId_o', 'displayName_o', 'x_o', 'y_o', 'speed_o',
       'acceleration_o', 'orientation_o', 'dir_o', 'jerseyNumber_o',
       'position_o', 'frameId', 'team_o', 'playDirection', 'route'],
      dtype='object')

# Merge ball tracking data with plays_expanded:

In [31]:
def football_filter(df):
    """
    Function for filtering ball release frame data for only ball tracking data
    :param df: Pandas dataframe containing tracking data by frame filtered for ball release event
    :return: Pandas Dataframe containing frames corresponding to ball tracking data
    """
    df = df[df.displayName.isin(['Football'])]
    return df

football_df = football_filter(release_df)

# TODO: Scott write some comments here about how this works
plays_expanded = pd.merge(plays_expanded, football_df, how='left', on=['gameId', 'playId'], suffixes=('', '_y'))
plays_expanded.drop(plays_expanded.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
plays_expanded.drop(columns = ['team', 'time', 'dis', 'event', 'o', 'dir', 'nflId', 'jerseyNumber', 'position', 'displayName'], inplace=True)
plays_expanded.rename(columns = {'x': 'x_b', 'y': 'y_b', 's': 'speed_b', 'a': 'acceleration_b'}, inplace=True)
plays_expanded.iloc[0]

gameId                   2018090600
playId                           75
down                              1
yardsToGo                        15
penaltyCodes                    NaN
penaltyJerseyNumbers            NaN
passResult                        C
offensePlayResult                10
playResult                       10
epa                        0.261827
nflId_o                 2.49545e+06
displayName_o           Julio Jones
x_o                           39.31
y_o                            8.42
speed_o                        3.11
acceleration_o                 6.67
orientation_o                335.11
dir_o                        117.88
jerseyNumber_o                   11
position_o                       WR
frameId                          36
team_o                         away
playDirection                  left
route                         HITCH
x_b                            23.1
y_b                           26.36
speed_b                        2.05
acceleration_b              

# Calculate DTR (Distance to Targeted Receiver) at time of release for each defender:

In [32]:
temp_df = []
import math

def find_DTR_at_release():
    """
    Calculates DTR (Distance to Targeted Receiver) for each defender at the time of release
    :return: Returns dataframe of defenders and their distances to the targeted receiver at time of release
    """
    # Filter for defensive positions only
    def_pos = ['DL', 'FS', 'DB', 'DE', 'OLB', 'SS', 'CB', 'ILB', 'S', 'MLB', 'NT', 'LB']
    
    temp_df = pd.merge(plays_expanded, release_df, how='right', on = ['gameId', 'playId'], suffixes=('', '_y'))
    temp_df.drop(plays_expanded.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
    temp_df = temp_df[temp_df.position.isin(def_pos)]
    
    # Calculate DTR for each defender
    temp_df['distance_from_receiver'] = (((temp_df['x']-temp_df['x_o'])**2)+((temp_df['y']-temp_df['y_o'])**2))**(1/2)
    
    # Create new dataframe consisting only of relevant defender distance data
    DTR_release_df = pd.DataFrame()
    DTR_release_df['defender_id'] = temp_df['nflId']
    DTR_release_df['defender_name'] = temp_df['displayName']
    DTR_release_df['defender_dist_to_target_receiver'] = temp_df['distance_from_receiver']
    DTR_release_df['gameId'] = temp_df['gameId']
    DTR_release_df['playId'] = temp_df['playId']
    DTR_release_df['frameId'] = temp_df['frameId']
    
    return temp_df, DTR_release_df
    
temp_df, DTR_release_df = find_DTR_at_release()
display(DTR_release_df)

,defender_id,defender_name,defender_dist_to_target_receiver,gameId,playId,frameId
1,79848.0,Malcolm Jenkins,7.768739,2018090600,75,36.0
3,2495613.0,Corey Graham,26.805257,2018090600,75,36.0
5,2534832.0,Rodney McLeod,17.920201,2018090600,75,36.0
7,2552315.0,Jordan Hicks,12.600496,2018090600,75,36.0
8,2552689.0,Ronald Darby,24.937933,2018090600,75,36.0
...,...,...,...,...,...,...
260710,2543811.0,Pierre Desir,17.223371,2018123011,4261,37.0
260713,2557961.0,Quincy Wilson,18.854013,2018123011,4261,37.0
260714,2558060.0,Malik Hooker,19.072226,2018123011,4261,37.0
260716,2558976.0,Kenny Moore,9.714469,2018123011,4261,37.0


In [33]:
# DEBUG: Display DTR (ascending) per ball release frame for all defenders in chronological order
display(DTR_release_df.sort_values(['gameId', 'playId', 'frameId', 'defender_dist_to_target_receiver']))

# Keep only the closest defender by DTR per ball release frame (at moment of release)
closest_DTR_release_df = DTR_release_df.sort_values(['gameId', 'playId', 'frameId', 'defender_dist_to_target_receiver']).drop_duplicates(subset=['gameId', 'playId', 'frameId'], keep='first')
closest_DTR_release_df = closest_DTR_release_df.reset_index(drop=True)
display(closest_DTR_release_df)

,defender_id,defender_name,defender_dist_to_target_receiver,gameId,playId,frameId
9,2555383.0,Jalen Mills,4.160048,2018090600,75,36.0
1,79848.0,Malcolm Jenkins,7.768739,2018090600,75,36.0
7,2552315.0,Jordan Hicks,12.600496,2018090600,75,36.0
5,2534832.0,Rodney McLeod,17.920201,2018090600,75,36.0
11,2558175.0,Nate Gerry,19.077783,2018090600,75,36.0
...,...,...,...,...,...,...
259585,2543830.0,Tre Boston,14.588533,2018123015,4104,52.0
259584,2543769.0,Deone Bucannon,16.077195,2018123015,4104,52.0
259579,2495504.0,Patrick Peterson,26.864017,2018123015,4104,52.0
259588,2557887.0,Haason Reddick,38.277279,2018123015,4104,52.0


,defender_id,defender_name,defender_dist_to_target_receiver,gameId,playId,frameId
0,2555383.0,Jalen Mills,4.160048,2018090600,75,36.0
1,79848.0,Malcolm Jenkins,4.243230,2018090600,146,31.0
2,2557958.0,Sidney Jones,5.338352,2018090600,168,34.0
3,79848.0,Malcolm Jenkins,4.243218,2018090600,190,39.0
4,2558175.0,Nate Gerry,2.124382,2018090600,256,31.0
...,...,...,...,...,...,...
17737,2560728.0,Tre Flowers,0.664831,2018123015,3794,36.0
17738,2560728.0,Tre Flowers,3.748333,2018123015,3819,42.0
17739,2560728.0,Tre Flowers,1.543826,2018123015,3969,45.0
17740,2543769.0,Deone Bucannon,8.079530,2018123015,4057,37.0


# Merge closest defender DTR at release data into plays_expanded:

In [34]:
# TODO: Write comments explaining this?
plays_expanded = pd.merge(plays_expanded, closest_DTR_release_df, how='left', on=['gameId', 'playId', 'frameId'])
plays_expanded = pd.merge(plays_expanded, release_df, how='left', right_on=['gameId', 'playId', 'nflId'], left_on=['gameId', 'playId', 'defender_id'], suffixes=('', '_y'))

plays_expanded.drop(plays_expanded.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
plays_expanded.drop(columns = ['time', 'dis', 'event', 'nflId'], inplace=True)

plays_expanded.rename(columns = {'team': 'team_d', 'displayName': 'displayName_d', 'x': 'x_d', 'y': 'y_d', 's': 'speed_d', 'a': 'acceleration_d', 'o': 'orientation_d', 'dir': 'dir_d', 'jerseyNumber': 'jerseyNumber_d', 'position': 'position_d'}, inplace = True)
plays_expanded.iloc[0]

gameId                               2018090600
playId                                       75
down                                          1
yardsToGo                                    15
penaltyCodes                                NaN
penaltyJerseyNumbers                        NaN
passResult                                    C
offensePlayResult                            10
playResult                                   10
epa                                    0.261827
nflId_o                             2.49545e+06
displayName_o                       Julio Jones
x_o                                       39.31
y_o                                        8.42
speed_o                                    3.11
acceleration_o                             6.67
orientation_o                            335.11
dir_o                                    117.88
jerseyNumber_o                               11
position_o                                   WR
frameId                                 

# Calculate DTR for defenders on ball arrival (as opposed to release):

In [35]:
# DEBUG FUNCTION TO HELP VIEW ALL INFO FOR A FRAME 
import numpy as np

def display_info(df, game_id, play_id, frame_id):
    frame_indices = np.where((df['gameId'] == game_id) & (df['playId'] == play_id) & (df['frameId'] == frame_id))
    display(df.loc[frame_indices])

display_info(tracking_df, 2018102110, 1710, 1)

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,gameId,playId,playDirection,route
7556967,2018-10-21T21:39:29.200Z,47.53,24.68,0.01,0.01,0.01,255.18,281.25,None,496937.0,Sean Lee,50.0,ILB,1,away,2018102110,1710,right,NaN
7556968,2018-10-21T21:39:29.200Z,41.10,23.80,0.19,0.06,0.02,78.91,93.41,None,2506340.0,Alex Smith,11.0,QB,1,home,2018102110,1710,right,NaN
7556969,2018-10-21T21:39:29.200Z,44.09,9.05,0.00,0.00,0.00,90.64,255.29,None,2532841.0,Michael Floyd,17.0,WR,1,home,2018102110,1710,right,POST
7556970,2018-10-21T21:39:29.200Z,43.82,18.81,0.00,0.00,0.00,101.64,59.93,None,2540160.0,Jordan Reed,86.0,TE,1,home,2018102110,1710,right,HITCH
7556971,2018-10-21T21:39:29.200Z,52.58,19.02,0.59,0.23,0.06,293.21,330.58,None,2541832.0,Jeff Heath,38.0,SS,1,away,2018102110,1710,right,NaN
7556972,2018-10-21T21:39:29.200Z,43.52,22.83,0.07,0.06,0.00,85.76,192.56,None,2550542.0,Kapri Bibbs,46.0,RB,1,home,2018102110,1710,right,NaN
7556973,2018-10-21T21:39:29.200Z,47.31,41.83,0.16,0.11,0.02,256.40,282.25,None,2552568.0,Byron Jones,31.0,CB,1,away,2018102110,1710,right,NaN
7556974,2018-10-21T21:39:29.200Z,46.94,23.10,0.53,0.29,0.04,259.49,281.98,None,2555310.0,Jaylon Smith,54.0,MLB,1,away,2018102110,1710,right,NaN
7556975,2018-10-21T21:39:29.200Z,44.51,42.03,0.01,0.01,0.00,98.09,117.03,None,2555343.0,Josh Doctson,18.0,WR,1,home,2018102110,1710,right,HITCH
7556976,2018-10-21T21:39:29.200Z,47.31,15.36,0.10,0.47,0.01,303.74,264.35,None,2555531.0,Anthony Brown,30.0,CB,1,away,2018102110,1710,right,NaN


# Pre-processing the data

Every 'pass_outcome_complete' play should result in a 'pass_arrived' event, but some 'pass_outcome_incomplete' plays result in a 'pass_arrived' event never being triggered because the ball was never passed close enough to the receiver. As a result, we want to consider the frame at which 'pass_outcome_incomplete' is triggered as the 'pass_arrived' event frame. For plays where 'pass_arrived' and 'pass_outcome_incomplete' are both triggered (maybe at different frames), we only want to consider the 'pass_arrived' event.

TODO: Explain more on why we only consider 'pass_arrived' as the important event.
        List the three cases out (pass complete, pass incomplete and arrived, pass incomplete but not arrived)

In [43]:
def drop_duplicate_arrive_incomplete(df):
    """
    Handles the case where 'pass_arrived' and 'pass_outcome_incomplete' occur on the same play (we only want to assess the game state at the point of 'pass arrived').
    Sort by player, then gameId, then playId to group any events occurring to the same player on the same play and then event ('pass_arrived' takes precedence over 'pass_outcome_incomplete')
    If there are duplicate rows (considering only the same player in the same game in the same play), then it must be 'pass_arrived' and 'pass_outcome_incomplete' events, and we remove the second one since it will be 'pass_outcome_incomplete'.
    :param df: Dataframe containing tracking player-frame data filtered for only player-frames with 'pass_arrived' or 'pass_outcome_incomplete' events.
    :return: Returns dataframe of tracking player-frame data filtered to only consider player-frames 'pass_arrived' events if a 'pass_outcome_incomplete' event occurs on the same play. 
    """
    # Sanity checks
    display(df.sort_values(['nflId', 'gameId','playId']))
    # Displays all the duplicated rows
    display(df[df.duplicated(['gameId', 'playId', 'nflId'], keep=False)].sort_values(['nflId', 'gameId','playId']))
    
    # Remove duplicates after sorting
    fixed_df = df.sort_values(['nflId', 'gameId','playId','event']).drop_duplicates(subset=['gameId', 'playId', 'nflId'], keep='first')
    
    # Sanity check, we should see both types at beginning and end hopefully
    display(fixed_df.sort_values('event'))
    
    return fixed_df


def drop_fumble_plays(tracking_df, pass_arrived_df):
    """
    Drop plays where the ball is fumbled and recovered by the defense.
    :param tracking_df: Dataframe containing tracking player-frame data (used to identify plays with 'fumble_defense_recovered' events)
    :param fixed_df: Dataframe containing tracking player-frame data with only 'pass_arrived' events (remove the fumble events from this)
    :return: Returns dataframe of tracking player-frame data filtered to not consider plays where the fumble is recovered by the defense. 
    """
    tracking_df = filter_tracking_frames_by_event(tracking_df, ['fumble_defense_recovered'])
    
    # Sanity check
    display(tracking_df.sort_values(['playId']))
    
    # Find playIds in which a player-frame with a fumble occurs and don't consider it
    play_ids = pd.Series(tracking_df['playId']).unique()
    frame_indices = ~pass_arrived_df['playId'].isin(play_ids)
    
    return pass_arrived_df[frame_indices]
    

# Filter all tracking data frames for 'pass_arrived' events and 'pass_outcome_incomplete' events
pass_arrived_incomplete_df = filter_tracking_frames_by_event(tracking_df, ['pass_arrived', 'pass_outcome_incomplete'])

# Sanity check (don't delete)
display(pass_arrived_incomplete_df)
# print(set(pass_arrived_incomplete_df['event']))

# For any rows/frames which have a 'pass_arrived' event and a 'pass_outcome_incomplete' event occur on the same play, drop the frame in which the 'pass_outcome_incomplete' event occurs
pass_arrived_df = drop_duplicate_arrive_incomplete(pass_arrived_incomplete_df)

# Convert all 'pass_outcome_incomplete' events to 'pass_arrived' events
pass_arrived_df.loc[pass_arrived_df.event == 'pass_outcome_incomplete', 'event'] = 'pass_arrived'

# Sanity check (don't delete)
display(pass_arrived_df)
#print(set(fixed_df['event']))

# Drop player-frames corresponding to plays in which a fumble recovered by defense occurs
pass_arrived_df = drop_fumble_plays(tracking_df, pass_arrived_df)
display(pass_arrived_df)

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,gameId,playId,playDirection,route
644,2018-09-07T01:07:19.200Z,23.70,25.81,0.59,0.51,0.06,153.07,181.23,pass_arrived,310.0,Matt Ryan,2.0,QB,47,away,2018090600,75,left,NaN
645,2018-09-07T01:07:19.200Z,37.66,8.15,7.57,1.72,0.75,174.68,173.52,pass_arrived,79848.0,Malcolm Jenkins,27.0,SS,47,home,2018090600,75,left,NaN
646,2018-09-07T01:07:19.200Z,38.89,5.46,4.23,1.63,0.42,233.31,202.58,pass_arrived,2495454.0,Julio Jones,11.0,WR,47,away,2018090600,75,left,HITCH
647,2018-09-07T01:07:19.200Z,37.10,35.71,3.64,4.01,0.36,193.28,119.37,pass_arrived,2495613.0,Corey Graham,24.0,FS,47,home,2018090600,75,left,NaN
648,2018-09-07T01:07:19.200Z,42.34,17.81,2.99,3.53,0.31,258.88,125.48,pass_arrived,2533040.0,Mohamed Sanu,12.0,WR,47,away,2018090600,75,left,HITCH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18308571,2018-12-31T04:23:18.400Z,54.42,33.74,6.32,4.19,0.64,195.30,137.73,pass_arrived,2558942.0,Anthony Firkser,86.0,TE,44,home,2018123011,4261,left,POST
18308572,2018-12-31T04:23:18.400Z,50.06,19.09,4.76,4.02,0.46,306.90,17.14,pass_arrived,2558976.0,Kenny Moore,23.0,CB,44,away,2018123011,4261,left,NaN
18308573,2018-12-31T04:23:18.400Z,46.49,28.06,4.53,1.37,0.45,334.75,326.70,pass_arrived,2560789.0,Darius Leonard,53.0,LB,44,away,2018123011,4261,left,NaN
18308574,2018-12-31T04:23:18.400Z,49.76,48.00,6.12,3.57,0.62,295.53,3.96,pass_arrived,2561521.0,Cameron Batson,17.0,WR,44,home,2018123011,4261,left,OUT


,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,gameId,playId,playDirection,route
18139954,2018-12-30T23:47:45.299Z,24.74,28.17,1.49,0.96,0.15,115.05,202.37,pass_arrived,252.0,Chad Henne,4.0,QB,38,home,2018123004,3370,right,NaN
18146374,2018-12-30T23:50:04.099Z,53.10,29.01,1.28,0.46,0.13,140.62,184.82,pass_arrived,252.0,Chad Henne,4.0,QB,33,home,2018123004,3439,right,NaN
18170905,2018-12-30T23:58:39.599Z,76.12,28.93,1.07,1.41,0.11,111.74,182.70,pass_arrived,252.0,Chad Henne,4.0,QB,35,home,2018123004,3642,right,NaN
18170961,2018-12-30T23:58:40.000Z,76.01,28.50,1.19,0.48,0.12,133.07,200.26,pass_outcome_incomplete,252.0,Chad Henne,4.0,QB,39,home,2018123004,3642,right,NaN
14310729,2018-12-09T20:05:56.400Z,34.18,27.98,1.24,0.75,0.12,25.41,197.74,pass_arrived,264.0,Josh Johnson,8.0,QB,43,home,2018120908,2841,right,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202095,2018-12-31T00:11:22.000Z,65.43,35.09,0.80,0.03,1.62,NaN,NaN,pass_arrived,NaN,Football,NaN,NaN,50,football,2018123015,3819,left,NaN
18220398,2018-12-31T00:18:02.599Z,97.93,24.68,19.01,7.54,2.18,NaN,NaN,pass_arrived,NaN,Football,NaN,NaN,57,football,2018123015,3969,left,NaN
18220440,2018-12-31T00:18:02.900Z,100.40,25.67,6.37,0.76,0.63,NaN,NaN,pass_outcome_incomplete,NaN,Football,NaN,NaN,60,football,2018123015,3969,left,NaN
18230620,2018-12-31T00:22:47.099Z,39.31,38.78,7.53,0.52,1.84,NaN,NaN,pass_arrived,NaN,Football,NaN,NaN,45,football,2018123015,4057,right,NaN


,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,gameId,playId,playDirection,route
18170905,2018-12-30T23:58:39.599Z,76.12,28.93,1.07,1.41,0.11,111.74,182.70,pass_arrived,252.0,Chad Henne,4.0,QB,35,home,2018123004,3642,right,NaN
18170961,2018-12-30T23:58:40.000Z,76.01,28.50,1.19,0.48,0.12,133.07,200.26,pass_outcome_incomplete,252.0,Chad Henne,4.0,QB,39,home,2018123004,3642,right,NaN
15506975,2018-12-16T19:12:53.299Z,34.72,25.59,1.01,1.39,0.08,146.58,67.52,pass_arrived,264.0,Josh Johnson,8.0,QB,50,away,2018121606,1699,right,NaN
15507029,2018-12-16T19:12:53.500Z,34.90,25.64,1.05,0.28,0.10,130.53,71.06,pass_outcome_incomplete,264.0,Josh Johnson,8.0,QB,52,away,2018121606,1699,right,NaN
15577306,2018-12-16T19:39:53.000Z,34.02,30.96,1.41,0.47,0.14,42.88,322.05,pass_arrived,264.0,Josh Johnson,8.0,QB,36,away,2018121606,2119,right,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18103541,2018-12-30T23:34:14.500Z,62.92,53.94,4.90,1.15,0.51,NaN,NaN,pass_outcome_incomplete,NaN,Football,NaN,NaN,65,football,2018123015,2903,right,NaN
18131840,2018-12-30T23:43:44.700Z,34.30,8.28,19.31,6.48,2.16,NaN,NaN,pass_arrived,NaN,Football,NaN,NaN,36,football,2018123015,3180,left,NaN
18131910,2018-12-30T23:43:45.200Z,36.32,6.11,3.57,0.26,0.36,NaN,NaN,pass_outcome_incomplete,NaN,Football,NaN,NaN,41,football,2018123015,3180,left,NaN
18220398,2018-12-31T00:18:02.599Z,97.93,24.68,19.01,7.54,2.18,NaN,NaN,pass_arrived,NaN,Football,NaN,NaN,57,football,2018123015,3969,left,NaN


,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,gameId,playId,playDirection,route
18139954,2018-12-30T23:47:45.299Z,24.74,28.17,1.49,0.96,0.15,115.05,202.37,pass_arrived,252.0,Chad Henne,4.0,QB,38,home,2018123004,3370,right,NaN
7557719,2018-10-21T21:39:34.500Z,71.58,25.32,6.53,4.01,0.67,145.51,93.64,pass_arrived,2556404.0,Maurice Harris,13.0,WR,54,home,2018102110,1710,right,GO
7558727,2018-10-21T21:40:40.400Z,71.80,50.09,2.11,2.63,0.23,172.64,22.99,pass_arrived,2556404.0,Maurice Harris,13.0,WR,53,home,2018102110,1751,right,OUT
7561499,2018-10-21T21:43:12.799Z,81.11,22.05,0.96,0.75,0.09,113.40,116.69,pass_arrived,2556404.0,Maurice Harris,13.0,WR,59,home,2018102110,1812,right,HITCH
7565214,2018-10-21T21:44:45.299Z,117.15,6.16,3.66,1.78,0.36,8.05,339.04,pass_arrived,2556404.0,Maurice Harris,13.0,WR,90,home,2018102110,1857,right,GO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2505236,2018-09-23T17:55:58.799Z,79.15,27.94,5.70,2.74,0.58,314.04,57.75,pass_outcome_incomplete,2533056.0,Luke Kuechly,59.0,MLB,51,home,2018092302,1267,left,NaN
14724544,2018-12-09T23:04:09.000Z,70.84,42.34,3.75,2.84,0.39,130.33,82.87,pass_outcome_incomplete,2558062.0,Gareon Conley,21.0,CB,51,home,2018120913,2180,left,NaN
2493255,2018-09-23T17:53:47.900Z,53.32,18.03,8.48,0.81,0.86,227.78,204.85,pass_outcome_incomplete,2533056.0,Luke Kuechly,59.0,MLB,44,home,2018092302,1199,left,NaN
4838613,2018-10-07T17:52:38.599Z,52.56,18.54,2.46,3.26,0.26,265.52,187.66,pass_outcome_incomplete,2533056.0,Luke Kuechly,59.0,MLB,60,home,2018100701,1082,left,NaN


,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,gameId,playId,playDirection,route
18139954,2018-12-30T23:47:45.299Z,24.74,28.17,1.49,0.96,0.15,115.05,202.37,pass_arrived,252.0,Chad Henne,4.0,QB,38,home,2018123004,3370,right,NaN
18146374,2018-12-30T23:50:04.099Z,53.10,29.01,1.28,0.46,0.13,140.62,184.82,pass_arrived,252.0,Chad Henne,4.0,QB,33,home,2018123004,3439,right,NaN
18170905,2018-12-30T23:58:39.599Z,76.12,28.93,1.07,1.41,0.11,111.74,182.70,pass_arrived,252.0,Chad Henne,4.0,QB,35,home,2018123004,3642,right,NaN
14310729,2018-12-09T20:05:56.400Z,34.18,27.98,1.24,0.75,0.12,25.41,197.74,pass_arrived,264.0,Josh Johnson,8.0,QB,43,home,2018120908,2841,right,NaN
14313285,2018-12-09T20:08:04.500Z,43.44,23.61,0.96,0.67,0.10,38.10,228.27,pass_arrived,264.0,Josh Johnson,8.0,QB,59,home,2018120908,2916,right,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18196868,2018-12-31T00:10:43.200Z,46.38,30.55,19.82,6.10,1.57,NaN,NaN,pass_arrived,NaN,Football,NaN,NaN,44,football,2018123015,3794,left,NaN
18202095,2018-12-31T00:11:22.000Z,65.43,35.09,0.80,0.03,1.62,NaN,NaN,pass_arrived,NaN,Football,NaN,NaN,50,football,2018123015,3819,left,NaN
18220398,2018-12-31T00:18:02.599Z,97.93,24.68,19.01,7.54,2.18,NaN,NaN,pass_arrived,NaN,Football,NaN,NaN,57,football,2018123015,3969,left,NaN
18230620,2018-12-31T00:22:47.099Z,39.31,38.78,7.53,0.52,1.84,NaN,NaN,pass_arrived,NaN,Football,NaN,NaN,45,football,2018123015,4057,right,NaN


,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,gameId,playId,playDirection,route
13400460,2018-12-02T21:06:34.000Z,40.99,35.82,3.99,0.95,0.40,44.98,41.85,fumble_defense_recovered,2558125.0,Patrick Mahomes,15.0,QB,107,away,2018120210,68,right,NaN
13400463,2018-12-02T21:06:34.000Z,65.11,40.74,1.65,2.35,0.16,NaN,NaN,fumble_defense_recovered,NaN,Football,NaN,NaN,107,football,2018120210,68,right,NaN
13400462,2018-12-02T21:06:34.000Z,65.03,41.56,4.37,3.97,0.46,66.26,43.85,fumble_defense_recovered,2560819.0,Nick Nelson,23.0,CB,107,home,2018120210,68,right,NaN
13400461,2018-12-02T21:06:34.000Z,60.67,41.57,4.31,2.30,0.43,72.48,55.28,fumble_defense_recovered,2559054.0,Nicholas Morrow,50.0,ILB,107,home,2018120210,68,right,NaN
13400459,2018-12-02T21:06:34.000Z,59.90,40.04,0.26,0.58,0.02,226.83,315.58,fumble_defense_recovered,2558062.0,Gareon Conley,21.0,CB,107,home,2018120210,68,right,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597788,2018-09-09T20:52:05.200Z,73.14,23.17,8.57,1.48,0.86,70.04,119.34,fumble_defense_recovered,2550164.0,T.J. Carrie,38.0,CB,58,home,2018090901,5369,left,NaN
597787,2018-09-09T20:52:05.200Z,44.29,25.57,0.76,2.59,0.05,251.31,178.79,fumble_defense_recovered,2543720.0,Christian Kirksey,58.0,OLB,58,home,2018090901,5369,left,NaN
597786,2018-09-09T20:52:05.200Z,68.33,25.70,6.72,4.85,0.67,287.01,36.42,fumble_defense_recovered,2543637.0,Terrance Mitchell,39.0,CB,58,home,2018090901,5369,left,NaN
597793,2018-09-09T20:52:05.200Z,50.78,16.18,2.10,1.55,0.20,227.71,129.97,fumble_defense_recovered,2557978.0,James Conner,30.0,RB,58,away,2018090901,5369,left,NaN


,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,gameId,playId,playDirection,route
18139954,2018-12-30T23:47:45.299Z,24.74,28.17,1.49,0.96,0.15,115.05,202.37,pass_arrived,252.0,Chad Henne,4.0,QB,38,home,2018123004,3370,right,NaN
18146374,2018-12-30T23:50:04.099Z,53.10,29.01,1.28,0.46,0.13,140.62,184.82,pass_arrived,252.0,Chad Henne,4.0,QB,33,home,2018123004,3439,right,NaN
18170905,2018-12-30T23:58:39.599Z,76.12,28.93,1.07,1.41,0.11,111.74,182.70,pass_arrived,252.0,Chad Henne,4.0,QB,35,home,2018123004,3642,right,NaN
14310729,2018-12-09T20:05:56.400Z,34.18,27.98,1.24,0.75,0.12,25.41,197.74,pass_arrived,264.0,Josh Johnson,8.0,QB,43,home,2018120908,2841,right,NaN
14314975,2018-12-09T20:08:44.200Z,58.71,30.52,1.29,2.23,0.12,107.37,264.78,pass_arrived,264.0,Josh Johnson,8.0,QB,39,home,2018120908,2947,right,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18196868,2018-12-31T00:10:43.200Z,46.38,30.55,19.82,6.10,1.57,NaN,NaN,pass_arrived,NaN,Football,NaN,NaN,44,football,2018123015,3794,left,NaN
18202095,2018-12-31T00:11:22.000Z,65.43,35.09,0.80,0.03,1.62,NaN,NaN,pass_arrived,NaN,Football,NaN,NaN,50,football,2018123015,3819,left,NaN
18220398,2018-12-31T00:18:02.599Z,97.93,24.68,19.01,7.54,2.18,NaN,NaN,pass_arrived,NaN,Football,NaN,NaN,57,football,2018123015,3969,left,NaN
18230620,2018-12-31T00:22:47.099Z,39.31,38.78,7.53,0.52,1.84,NaN,NaN,pass_arrived,NaN,Football,NaN,NaN,45,football,2018123015,4057,right,NaN


In [45]:
def find_DTR_at_arrival(plays_expanded, df):
    """
    Calculates DTR (Distance to Targeted Receiver) for each defender at the time of arrival
    :return: Returns dataframe of defenders and their distances to the targeted receiver at time of arrival
    """
    temp_df = []
    
    def_pos = ['DL', 'FS', 'DB', 'DE', 'OLB', 'SS', 'CB', 'ILB', 'S', 'MLB', 'NT', 'LB']
    
    temp_df = pd.merge(plays_expanded, df, how='right', on = ['gameId', 'playId'], suffixes=('', '_y'))
    temp_df.drop(plays_expanded.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
    temp_df = temp_df[temp_df.position.isin(def_pos)]
    
    temp_df['distance_from_receiver'] = (((temp_df['x'] - temp_df['x_o']) ** 2)
                                        + ((temp_df['y'] - temp_df['y_o']) ** 2)
                                        ) ** (1/2)
    
    DTR_arrival_df = pd.DataFrame()
    DTR_arrival_df['defender_id'] = temp_df['nflId']
    DTR_arrival_df['defender_name'] = temp_df['displayName']
    DTR_arrival_df['defender_dist_to_target_receiver'] = temp_df['distance_from_receiver']
    DTR_arrival_df['gameId'] = temp_df['gameId']
    DTR_arrival_df['playId'] = temp_df['playId']
    DTR_arrival_df['frameId'] = temp_df['frameId']
    return temp_df, DTR_arrival_df

temp_df, DTR_arrival_df = find_DTR_at_arrival(plays_expanded, pass_arrived_df)
display(defenders_on_arrival_df)
display(DTR_arrival_df)

,gameId,playId,down,yardsToGo,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,...,event,nflId,displayName,jerseyNumber,position,frameId_y,team,playDirection_y,route_y,distance_from_receiver
2,2018123004,3370,2,11,NaN,NaN,C,22,22,1.967316,...,pass_arrived,2532986.0,Tahir Whitehead,59.0,OLB,38,away,right,NaN,12.212817
4,2018123004,3370,2,11,NaN,NaN,C,22,22,1.967316,...,pass_arrived,2541200.0,Rashaan Melvin,22.0,CB,38,away,right,NaN,10.128267
5,2018123004,3370,2,11,NaN,NaN,C,22,22,1.967316,...,pass_arrived,2555342.0,Karl Joseph,42.0,SS,38,away,right,NaN,13.014719
6,2018123004,3370,2,11,NaN,NaN,C,22,22,1.967316,...,pass_arrived,2555568.0,Erik Harris,25.0,SS,38,away,right,NaN,12.125090
7,2018123004,3370,2,11,NaN,NaN,C,22,22,1.967316,...,pass_arrived,2557972.0,Marquel Lee,55.0,MLB,38,away,right,NaN,2.530178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249634,2018110403,4041,1,10,NaN,NaN,C,9,9,-0.378102,...,pass_arrived,2555349.0,Kendall Fuller,23.0,CB,34,away,left,NaN,4.405553
249636,2018110403,4041,1,10,NaN,NaN,C,9,9,-0.378102,...,pass_arrived,2555542.0,Jordan Lucas,24.0,CB,34,away,left,NaN,25.499735
249639,2018110403,4041,1,10,NaN,NaN,C,9,9,-0.378102,...,pass_arrived,2560827.0,Dorian O'Daniel,44.0,LB,34,away,left,NaN,11.739199
249642,2018110403,4041,1,10,NaN,NaN,C,9,9,-0.378102,...,pass_arrived,2561103.0,Tremon Smith,39.0,DB,34,away,left,NaN,8.054098


,defender_id,defender_name,defender_dist_to_target_receiver,gameId,playId,frameId
2,2532986.0,Tahir Whitehead,12.212817,2018123004,3370,33.0
4,2541200.0,Rashaan Melvin,10.128267,2018123004,3370,33.0
5,2555342.0,Karl Joseph,13.014719,2018123004,3370,33.0
6,2555568.0,Erik Harris,12.125090,2018123004,3370,33.0
7,2557972.0,Marquel Lee,2.530178,2018123004,3370,33.0
...,...,...,...,...,...,...
249634,2555349.0,Kendall Fuller,4.405553,2018110403,4041,26.0
249636,2555542.0,Jordan Lucas,25.499735,2018110403,4041,26.0
249639,2560827.0,Dorian O'Daniel,11.739199,2018110403,4041,26.0
249642,2561103.0,Tremon Smith,8.054098,2018110403,4041,26.0


We want to get the closest three defenders to the targeted receiver at the moment of ball arrival who were NOT the closest defender to the targeted receiver at the moment of ball release.

TODO: Explain why this is the case some more.

In [54]:
# Get DTR (ascending) for all defenders at the moment of ball arrival for each play in chronological order
sorted_DTR_arrival_df = DTR_arrival_df.sort_values(['gameId', 'playId', 'frameId', 'defender_dist_to_target_receiver'])
display(sorted_DTR_arrival_df)

# DEBUG: Display DTR (ascending) for all defenders at the moment of release for each play in chronological order
display(DTR_release_df.sort_values(['gameId', 'playId', 'frameId', 'defender_dist_to_target_receiver']))

# Get the closest defender by DTR at the moment of release for each play in chronological order
sorted_closest_DTR_release_df = closest_DTR_release_df.sort_values(['gameId', 'playId', 'frameId', 'defender_dist_to_target_receiver'])
display(sorted_closest_DTR_release_df)

# Append the list of closest defenders by DTR at the moment of release for each play to the list of all defenders at the moment of arrival for each play
# Then remove any duplicates (both of them) in order to ensure we get the closest three defenders at arrival time who were NOT the closest defender at release time
sorted_closest_defenders_arrival_with_exclusion_df = sorted_DTR_arrival_df.append(sorted_closest_DTR_release_df).drop_duplicates(subset=['gameId', 'playId', 'frameId', 'defender_id'],keep=False)
display(sorted_closest_defenders_arrival_with_exclusion_df)

# Group defenders by each play
grouped_df = sorted_closest_defenders_arrival_with_exclusion_df.groupby(['gameId', 'playId', 'frameId']) #as_index=False

# DEBUG: View groups representing each frame formed by players sorted in ascending order of DTR
# count = 0
# for key, item in grouped_df:
# #     print(key)
# #     print(item)
#     if len(item) == 1:
#         print(key)
#         print(item)
# #     if count > 5: break
# #     else: count += 1


# TODO: CHECK CORRECTNESS
# Keep only the three closest defenders by DTR for each play at the moment of arrival
top_three_defenders_arrival_with_exclusion = grouped_df.head(3)
display(top_three_defenders_arrival_with_exclusion)

,defender_id,defender_name,defender_dist_to_target_receiver,gameId,playId,frameId
2144,79848.0,Malcolm Jenkins,1.671945,2018090600,75,36.0
2152,2555383.0,Jalen Mills,6.663640,2018090600,75,36.0
2150,2552315.0,Jordan Hicks,8.476633,2018090600,75,36.0
2154,2558175.0,Nate Gerry,13.860307,2018090600,75,36.0
2148,2534832.0,Rodney McLeod,14.310853,2018090600,75,36.0
...,...,...,...,...,...,...
140536,2543830.0,Tre Boston,11.133822,2018123015,4104,52.0
140535,2543769.0,Deone Bucannon,11.744820,2018123015,4104,52.0
140530,2495504.0,Patrick Peterson,17.264058,2018123015,4104,52.0
140540,2558009.0,Budda Baker,36.022983,2018123015,4104,52.0


,defender_id,defender_name,defender_dist_to_target_receiver,gameId,playId,frameId
9,2555383.0,Jalen Mills,4.160048,2018090600,75,36.0
1,79848.0,Malcolm Jenkins,7.768739,2018090600,75,36.0
7,2552315.0,Jordan Hicks,12.600496,2018090600,75,36.0
5,2534832.0,Rodney McLeod,17.920201,2018090600,75,36.0
11,2558175.0,Nate Gerry,19.077783,2018090600,75,36.0
...,...,...,...,...,...,...
259585,2543830.0,Tre Boston,14.588533,2018123015,4104,52.0
259584,2543769.0,Deone Bucannon,16.077195,2018123015,4104,52.0
259579,2495504.0,Patrick Peterson,26.864017,2018123015,4104,52.0
259588,2557887.0,Haason Reddick,38.277279,2018123015,4104,52.0


,defender_id,defender_name,defender_dist_to_target_receiver,gameId,playId,frameId
0,2555383.0,Jalen Mills,4.160048,2018090600,75,36.0
1,79848.0,Malcolm Jenkins,4.243230,2018090600,146,31.0
2,2557958.0,Sidney Jones,5.338352,2018090600,168,34.0
3,79848.0,Malcolm Jenkins,4.243218,2018090600,190,39.0
4,2558175.0,Nate Gerry,2.124382,2018090600,256,31.0
...,...,...,...,...,...,...
17737,2560728.0,Tre Flowers,0.664831,2018123015,3794,36.0
17738,2560728.0,Tre Flowers,3.748333,2018123015,3819,42.0
17739,2560728.0,Tre Flowers,1.543826,2018123015,3969,45.0
17740,2543769.0,Deone Bucannon,8.079530,2018123015,4057,37.0


,defender_id,defender_name,defender_dist_to_target_receiver,gameId,playId,frameId
2144,79848.0,Malcolm Jenkins,1.671945,2018090600,75,36.0
2150,2552315.0,Jordan Hicks,8.476633,2018090600,75,36.0
2154,2558175.0,Nate Gerry,13.860307,2018090600,75,36.0
2148,2534832.0,Rodney McLeod,14.310853,2018090600,75,36.0
2146,2495613.0,Corey Graham,27.379339,2018090600,75,36.0
...,...,...,...,...,...,...
17591,2552313.0,Hayes Pullard,1.180678,2018123013,1930,29.0
17613,2560946.0,Isaac Yiadom,2.236001,2018123013,3610,43.0
17627,1302.0,Aqib Talib,3.430714,2018123014,329,37.0
17635,2532789.0,Mark Barron,1.147388,2018123014,746,39.0


(2018090600, 1295, 35.0)
    defender_id defender_name  defender_dist_to_target_receiver      gameId  \
21    2557967.0    Duke Riley                          0.886454  2018090600   

    playId  frameId  
21    1295     35.0  
(2018090600, 2693, 41.0)
    defender_id defender_name  defender_dist_to_target_receiver      gameId  \
49    2555162.0   Deion Jones                          1.393018  2018090600   

    playId  frameId  
49    2693     41.0  
(2018090600, 3095, 43.0)
    defender_id    defender_name  defender_dist_to_target_receiver  \
56    2539334.0  Desmond Trufant                          2.490462   

        gameId  playId  frameId  
56  2018090600    3095     43.0  
(2018090900, 2646, 42.0)
     defender_id      defender_name  defender_dist_to_target_receiver  \
128    2506268.0  Lorenzo Alexander                          8.514987   

         gameId  playId  frameId  
128  2018090900    2646     42.0  
(2018090900, 2732, 28.0)
     defender_id defender_name  defender_di

...

(2018120202, 2693, 41.0)
       defender_id  defender_name  defender_dist_to_target_receiver  \
12498    2552488.0  Marcus Peters                          1.155206   

           gameId  playId  frameId  
12498  2018120202    2693     41.0  
(2018120203, 1806, 29.0)
       defender_id   defender_name  defender_dist_to_target_receiver  \
12561    2495807.0  Antoine Bethea                          4.303359   

           gameId  playId  frameId  
12561  2018120203    1806     29.0  
(2018120203, 2190, 44.0)
       defender_id defender_name  defender_dist_to_target_receiver  \
12566    2558009.0   Budda Baker                          1.547966   

           gameId  playId  frameId  
12566  2018120203    2190     44.0  
(2018120203, 2370, 42.0)
       defender_id defender_name  defender_dist_to_target_receiver  \
12571    2560755.0  Josh Jackson                          2.425283   

           gameId  playId  frameId  
12571  2018120203    2370     42.0  
(2018120203, 2933, 44.0)
       de

       defender_id defender_name  defender_dist_to_target_receiver  \
13457    2558005.0   Jayon Brown                          5.552018   

           gameId  playId  frameId  
13457  2018120600    2818     38.0  
(2018120900, 275, 37.0)
       defender_id     defender_name  defender_dist_to_target_receiver  \
13485    2543597.0  Avery Williamson                          5.250238   

           gameId  playId  frameId  
13485  2018120900     275     37.0  
(2018120900, 1078, 27.0)
       defender_id     defender_name  defender_dist_to_target_receiver  \
13493    2560713.0  Tremaine Edmunds                         10.413165   

           gameId  playId  frameId  
13493  2018120900    1078     27.0  
(2018120900, 1892, 48.0)
       defender_id defender_name  defender_dist_to_target_receiver  \
13503    2560914.0  Levi Wallace                          1.042929   

           gameId  playId  frameId  
13503  2018120900    1892     48.0  
(2018120900, 2646, 39.0)
       defender_id     de

(2018121602, 2084, 49.0)
       defender_id defender_name  defender_dist_to_target_receiver  \
14924    2539290.0  Jordan Poyer                          2.657066   

           gameId  playId  frameId  
14924  2018121602    2084     49.0  
(2018121602, 3770, 100.0)
       defender_id defender_name  defender_dist_to_target_receiver  \
14951    2543872.0  Nevin Lawson                          1.242578   

           gameId  playId  frameId  
14951  2018121602    3770    100.0  
(2018121603, 2865, 35.0)
       defender_id     defender_name  defender_dist_to_target_receiver  \
14999    2543571.0  Bashaud Breeland                          1.338693   

           gameId  playId  frameId  
14999  2018121603    2865     35.0  
(2018121603, 2992, 30.0)
       defender_id defender_name  defender_dist_to_target_receiver  \
15001    2558119.0    Josh Jones                          2.324995   

           gameId  playId  frameId  
15001  2018121603    2992     30.0  
(2018121605, 2433, 34.0)
      

       defender_id defender_name  defender_dist_to_target_receiver  \
16785    2495775.0   Eric Weddle                          2.673668   

           gameId  playId  frameId  
16785  2018123000    3916     27.0  
(2018123000, 4196, 52.0)
       defender_id defender_name  defender_dist_to_target_receiver  \
16789       4365.0  Brandon Carr                          1.408723   

           gameId  playId  frameId  
16789  2018123000    4196     52.0  
(2018123000, 4376, 31.0)
       defender_id defender_name  defender_dist_to_target_receiver  \
16795    2495775.0   Eric Weddle                          4.611833   

           gameId  playId  frameId  
16795  2018123000    4376     31.0  
(2018123001, 1155, 27.0)
       defender_id      defender_name  defender_dist_to_target_receiver  \
16810    2506268.0  Lorenzo Alexander                          4.410045   

           gameId  playId  frameId  
16810  2018123001    1155     27.0  
(2018123002, 1069, 31.0)
       defender_id defender_na

,defender_id,defender_name,defender_dist_to_target_receiver,gameId,playId,frameId
2144,79848.0,Malcolm Jenkins,1.671945,2018090600,75,36.0
2150,2552315.0,Jordan Hicks,8.476633,2018090600,75,36.0
2154,2558175.0,Nate Gerry,13.860307,2018090600,75,36.0
2161,2552315.0,Jordan Hicks,2.310368,2018090600,146,31.0
2164,2555383.0,Jalen Mills,2.446753,2018090600,146,31.0
...,...,...,...,...,...,...
17591,2552313.0,Hayes Pullard,1.180678,2018123013,1930,29.0
17613,2560946.0,Isaac Yiadom,2.236001,2018123013,3610,43.0
17627,1302.0,Aqib Talib,3.430714,2018123014,329,37.0
17635,2532789.0,Mark Barron,1.147388,2018123014,746,39.0


# Adding receiver and ball at arrival

In [95]:
def arrival_filter(df):
    arrival = df['event'] == 'pass_arrived'
    return df[arrival]

arrival_df = arrival_filter(tracking_df)

plays_expanded = pd.merge(plays_expanded, arrival_df, how='left', left_on = ['gameId', 'playId', 'nflId_o'], right_on = ['gameId', 'playId', 'nflId'],suffixes=('', '_y'))
plays_expanded.drop(plays_expanded.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
plays_expanded.drop(columns = ['time', 'dis', 'event', 'team', 'displayName', 'jerseyNumber', 'position', 'nflId'], inplace = True)
plays_expanded.rename(columns = {'x': 'x_o_a', 'y': 'y_o_a', 's': 'speed_o_a', 'a': 'acceleration_o_a', 'o': 'orientation_o_a', 'dir': 'dir_o_a'}, inplace = True)
plays_expanded.iloc[0]

gameId                    2018090600
playId                            75
down                               1
yardsToGo                         15
penaltyCodes                     NaN
penaltyJerseyNumbers             NaN
passResult                         C
offensePlayResult                 10
playResult                        10
epa                         0.261827
nflId_o                  2.49545e+06
displayName_o            Julio Jones
x_o                            39.31
y_o                             8.42
speed_o                         3.11
acceleration_o                  6.67
orientation_o                 335.11
dir_o                         117.88
jerseyNumber_o                    11
position_o                        WR
frameId                           36
team_o                          away
playDirection                   left
route                          HITCH
x_b                             23.1
y_b                            26.36
speed_b                         2.05
a

In [96]:
def football_arrival_filter(df):
    football_arrival_df = df[df.displayName.isin(['Football'])]
    return football_arrival_df

football_arrival_df = football_filter(arrival_df)

plays_expanded = pd.merge(plays_expanded, football_arrival_df, how='left', on = ['gameId', 'playId'], suffixes=('', '_y'))
plays_expanded.drop(plays_expanded.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
plays_expanded.drop(columns = ['team', 'time', 'dis', 'event', 'o', 'dir', 'nflId', 'jerseyNumber', 'position', 'displayName'], inplace = True)
plays_expanded.rename(columns = {'x': 'x_b_a', 'y': 'y_b_a', 's': 'speed_b_a', 'a': 'acceleration_b_a'}, inplace = True)
plays_expanded.loc[0]

gameId                    2018090600
playId                            75
down                               1
yardsToGo                         15
penaltyCodes                     NaN
penaltyJerseyNumbers             NaN
passResult                         C
offensePlayResult                 10
playResult                        10
epa                         0.261827
nflId_o                  2.49545e+06
displayName_o            Julio Jones
x_o                            39.31
y_o                             8.42
speed_o                         3.11
acceleration_o                  6.67
orientation_o                 335.11
dir_o                         117.88
jerseyNumber_o                    11
position_o                        WR
frameId                           36
team_o                          away
playDirection                   left
route                          HITCH
x_b                             23.1
y_b                            26.36
speed_b                         2.05
a